# Perform univariate analyses exploring preprocessing decisions

This code should execute and recreate the main images reported in the manuscript. The figures should be identical but the statistics may differ slightly due to the random nature of bootstrap resampling.

To make this script work, edit the file paths in the cell below to go to the two directories needed: 
1. The main path of the project directory (the cloned neuropipe repo)
2. The place where the data for the methods is stored (downloaded from Dryad)

If these are set up correctly then you should be able to run all cells

Table of contents:
>[Set up](#setup)  
>[Plot voxelwise results](#wholebrain)  
>[Plot ROI bar plots](#rois)  
>[Plot different preprocessing ROIs for different parameters](#explore)  


In [ ]:
# File paths
proj_path = './infant_neuropipe'
preloaded_path = '%s/data/methods_data/' % proj_path

## Set up<a id='setup'></a>

Run the cells below to set up the analysis. This sets up the modules in the first cell, defines the parameters for plotting in the second, and then makes all the needed functions in the third (this is a big cell, keep scrolling!). 

In [ ]:
import nibabel as nib
import numpy as np
from scipy import stats
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
import sys
import os
from nilearn import plotting
import scipy.spatial.distance as sp_distance
from scipy import stats, ndimage
from nilearn import datasets
from nilearn.input_data import NiftiMasker
import pandas as pd
import glob
from scipy.ndimage.morphology import binary_dilation
import matplotlib.style
import matplotlib as mpl
import nibabel
mpl.style.use('seaborn-poster')
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

In [ ]:
# Make additional paths
output_mask_path = proj_path + 'results/preprocessing_exploration/matplotlib_plots/'
mask_path = preloaded_path + '/ROIs/'

os.system('mkdir -p %s' % output_mask_path)

occipital_filename = mask_path + 'occipital_MNI_1mm.nii.gz'
A1_filename = mask_path + 'A1_MNI_1mm.nii.gz'
V1_filename = mask_path + 'V1_MNI_1mm.nii.gz'
LOC_filename = mask_path + 'LOC_MNI_1mm.nii.gz'
cut_coords=[19, -8]#[12, -9] # [17, 10]  # [23, -3]

V1_color = np.array((70, 157, 49)) / 255
LOC_color = np.array((38, 92, 124)) / 255
A1_color = np.array((102, 102, 102)) / 255

bar_plot_color = np.array((205, 152, 248)) / 255

heatmap_cmap = 'inferno'

#Make the ROI masks color maps
V1_color_map = ListedColormap(list(V1_color) + [0.5])
A1_color_map = ListedColormap(list(A1_color) + [0.5])
LOC_color_map = ListedColormap(list(LOC_color) + [0.5])

# Do you want to make nans go to zero, depicting it with a dotted line
set_nans_to_zero = 1

# Specify various plotting parameters
linecolor = [0.8, 0.8, 0.8]
face_color = LOC_color
face_color_alt = 'white'
linestyle = '-'
markeredgewidth = 2 # How think is the perimeter of the marker
linewidth = 1 # How wide should the lines be connecting points
bg_bar_color = [0.7, 0.7, 0.7]
markersize=7 # How big are the markers
bee_swarm_spacing = markersize * 1.1 # How far apart are the beeswarm markers
alpha = 0.5

# How do you want to categorize the age bands
age_bands = np.asarray([[0, 12], [12, 18], [18, np.inf]])


In [ ]:
# Convert a vector of brain voxels into a volume by unwrapping according to the mask
def vector_2_vol(mask_name,
                 brain_mask,
                vector,
                output_name):

    # Load in the brain data for reference
    brain_nii = nib.load(mask_name)
    vol = np.zeros(brain_nii.shape)

    # Get the list of nonzero voxel coordinates
    coords = np.where(brain_mask)

    # Map the ISC data into brain space
    vol[coords] = vector

    # Return or save the ISC data as a volume
    out_nifti = nib.Nifti1Image(vol, brain_nii.affine, brain_nii.header)
    if output_name is not None:
       nib.save(out_nifti, output_name)
    
    return out_nifti

# Convert the correlation into a p value
def corr2p(N, r):        
    se = 1 / np.sqrt(N - 3)  # The standard error of a fisher transform distribution
    p_val = stats.t.pdf(r / se, df=N - 2)
    return p_val

def plot_df_line(df, bar_color=bar_plot_color):
    # Plot the data frame based on the text file 
    
    # What are the column names
    conditions = df.columns[1:]
    
    # What is the data array
    data_array = df.values
    
    plt.figure()
    
    # Plot the data mean with a bar plot
    plt.bar(np.arange(len(conditions)), np.nanmean(data_array[:, 1:], 0), color=bar_color)
    plt.errorbar(np.arange(len(conditions)), np.nanmean(data_array[:, 1:], 0), np.nanstd(data_array[:, 1:], 0) / np.sqrt(data_array[:, 1:].shape[0]), linestyle='none', color=np.asarray(bar_color) * 0.75)
    
    # Set to zero so it is just a line plot
    markersize = 0
    
    # Plot the ppt data
    for ppt_counter in range(data_array.shape[0]):
        
        # Pull outdata for this condition
        data_ppt = data_array[ppt_counter, :]
        
        # What is the participant age
        age = data_ppt[0]
        
        if age > age_bands[0, 0] and age <= age_bands[0, 1]:
            fillstyle='none'
        elif age > age_bands[1, 0] and age <= age_bands[1, 1]:
            fillstyle='bottom'
        elif age > age_bands[2, 0] and age <= age_bands[2, 1]:
            fillstyle='full'
        
        # Pull out the condition data
        data_condition = data_array[ppt_counter, 1:]
        
        # Do you want to make NaNs go to zero (because they were excluded)
        if set_nans_to_zero == 1:
            plt.plot(np.nan_to_num(data_condition), linestyle='--', color='k', linewidth=linewidth / 2) # Put a dotted line beneath all plots so that it will continue if not show elsewhere

        # Plot the data
        if fillstyle is 'none':
            plt.plot(data_condition, color=linecolor, markeredgecolor=bg_bar_color, markerfacecolor='None', linestyle=linestyle, marker='o', markersize=markersize, markeredgewidth=markeredgewidth, linewidth=linewidth)
        else:
            plt.plot(data_condition, color=linecolor, markeredgecolor=bg_bar_color, markerfacecolor=bg_bar_color, linestyle=linestyle, marker='o', markersize=markersize, markerfacecoloralt='None', fillstyle=fillstyle, markeredgewidth=markeredgewidth, linewidth=linewidth)
    
    # Remove the box around the figure
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xticks(np.arange(len(conditions)), list(conditions), rotation=45)
    plt.xlim([-0.5, len(conditions) - 0.5])
     
#     # Run the one way anova on the data
#     arrays = []
#     for i in range(1, df.values.shape[1]):
#         arrays += [list(df.values[np.isnan(df.values[:, i]) == False, i])]
#     f_stats = stats.f_oneway(*arrays)

#     # Get the degrees of freedom
#     k = df.values.shape[1]
#     N = np.sum(np.isnan(df.values[:, 1:]) == False)
#     df_b = k - 1
#     df_w = N - k

#     print('F(%d,%d)=%0.3f, p=%0.3f' % (df_b, df_w, f_stats.statistic, f_stats.pvalue))

def plot_bar(df, colors, plot_SE=True):

    # What are the column names
    conditions = df.columns[1:]
    
    # What is the data array
    data_array = df.values
    
    plt.figure()
    
    # Plot the data mean with a bar plot
    plt.bar(np.arange(len(conditions)), np.nanmean(data_array[:, 1:], 0), color=colors)
    
    # Do you want to plot the SE?
    if plot_SE is True:
        
        error_ROI = np.nanstd(data_array[:, 1:], 0) / np.sqrt(data_array[:, 1:].shape[0])
        plt.errorbar(np.arange(len(conditions)), np.nanmean(data_array[:, 1:], 0), np.vstack((error_ROI, error_ROI)), linestyle='none', color=np.asarray(colors) * 0.5)
    else:
        
        # Plot each individual confound
        for roi_counter in range(1, data_array.shape[1]):
            lower_bound, upper_bound = randomise_diff(data_array[:, roi_counter], return_percentile=True)
            plt.errorbar(roi_counter - 1, np.nanmean(data_array[:, roi_counter], 0), np.vstack((lower_bound, upper_bound)), linestyle='none', color=np.asarray(colors[roi_counter - 1]) * 0.75)
        
        
    # Remove the box around the figure
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xticks(np.arange(len(conditions)), list(conditions), rotation=45)
    plt.xlim([-0.5, len(conditions) - 0.5])
    
    # Set the ylim depending on the type
    if 'proportion_sig_voxels' in DV:
        plt.ylim(y_range)
        plt.yticks(y_ticks)
        plt.ylabel('Prop. sig task vs rest')
        plt.hlines(1 - p_val, -1, 3, 'k', linestyles='dashed', linewidth=1)

        
def plot_column(df, ylim=[0, 1]):
    
    # Set the size
    plt.figure(figsize=(1.5, 2.4))
    
    # Cycle through participants
    conditions = df.columns[1:]
    for ppt_counter in range(df.shape[0]):
        
        # Pull out the ppt age
        age = df['Ppt_age'][ppt_counter]
    
        # Sort the ppts into age bins
        if age > age_bands[0, 0] and age <= age_bands[0, 1]:
            age_color = 0
        elif age > age_bands[1, 0] and age <= age_bands[1, 1]:
            age_color = 1 
        elif age > age_bands[2, 0] and age <= age_bands[2, 1]:
            age_color = 2 
            
        # Cycle through the ROIs
        ROI_vec = []
        for roi_counter, ROI in enumerate(conditions):
            ROI_vec += [df[ROI][ppt_counter]]
        
        # Plot the data acros ROIs
        plt.plot([0, 1, 2], ROI_vec, c='k', linewidth=0.5)
        
    # Remove the box around the figure
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xticks(np.arange(len(conditions)), list(conditions))
    plt.xticks([0, 1, 2], ['V1', 'LOC', 'A1'])
    plt.xlim([-0.5, len(conditions) - 0.5])
    
    # Plot the ylim if it is provided
    if len(ylim) > 0:
        plt.ylim(ylim)
        plt.yticks([0, 1.0])
        plt.hlines(1 - p_val, -1, 3, 'k', linestyles='dashed', linewidth=1)
    
    
def plot_sig_map(overall_task_based_name, output_name, threshold, N=None, ROIs=['V1', 'A1']):
    
    # Assume this will be 1 unless changed below
    vmax = 1
    
    # If the data is just the raw t stat then take the threshold as a p value and convert it into an uncorrected tstat threshold
    if (overall_task_based_name.find('tstat') > 0) and threshold < 1 :
        threshold = stats.t.ppf(threshold, N - 1)
        print('Setting t threshold to %0.2f' % threshold)

    vmax = 5
        
    # Load the participant in
    nii = nib.load(overall_task_based_name)
    
    # If you are using a p value threshold then change the range of values to show the full colormap
    if vmax == 1:
        overall_task_based = nii.get_data()

        # Pull out the data and do a hacky step to make the color range equivalent to the range from threshold to 1
        overall_task_based = (overall_task_based - threshold) * (1 / (1 - threshold))
        overall_task_based[overall_task_based<=0] = 0
        print('The range is actually %0.2f to 1' % threshold)

        nii = nib.Nifti1Image(overall_task_based.astype('float'), nii.affine)
        
        # Set to none since you have manually done this
        threshold = None

    fig = plotting.plot_stat_map(nii, 
                                 vmax=vmax, 
                                 cmap=heatmap_cmap,
                                 draw_cross=False,
                                 cut_coords=cut_coords,
                                 display_mode='xz',
                                 colorbar=True,
                                 threshold=threshold,
                                 symmetric_cbar=True,
                                )
    
    plt.savefig(output_name[:-4] + '_without_roi.svg')

    # Add the ROI to this
    #fig.add_overlay(occipital_filename, cmap=occipital_color_map)
    if 'V1' in ROIs:
        fig.add_contours(V1_filename, cmap=V1_color_map, linewidths=1, alpha=alpha)
    if 'A1' in ROIs:
        fig.add_contours(A1_filename, cmap=A1_color_map, linewidths=1, alpha=alpha)
    if 'LOC' in ROIs:
        fig.add_contours(LOC_filename, cmap=LOC_color_map, linewidths=1, alpha=alpha)
    
    fig.add_overlay(nii, 
                    threshold=threshold, 
                    cmap=heatmap_cmap,
                    vmax=vmax)
    plt.savefig(output_name)
     
        
def file_name_extractor(ppt_id, aggregate_type, feat_name):
    
    ppt_data = pd.read_csv(preloaded_path + 'participant_age.txt', header=None, sep='\t', names=['neuropipe_name', 'age', 'cohort'])
    cohort_ppts = []
    for ppt in ppt_data['neuropipe_name']:
        cohort = ppt_data[ppt_data['neuropipe_name'] == ppt]['cohort'].values[0]
        if cohort == ppt_id:
            cohort_ppts += [ppt_data[ppt_data['neuropipe_name'] == ppt]['neuropipe_name'].values[0]]

    if aggregate_type == 'concat':
        file_names = []
        for ppt in cohort_ppts:
            file_names += glob.glob('%s/stat_maps_session/%s/%s*.nii.gz' % (preloaded_path, feat_name, ppt))

    elif aggregate_type == 'all':
        file_names = []
        for ppt in cohort_ppts:
            file_names += glob.glob('%s/stat_maps_runs/%s/%s*.nii.gz' % (preloaded_path, feat_name, ppt))
            
    return file_names
        
    
def load_each_subject(feat_name, aggregate_type, ppt_stem, ROIs, DV):
    
    # Cycle through the ROIs
    df = pd.DataFrame()
    for ROI_counter, ROI in enumerate(ROIs):
    
        # Get the file names
        files = file_name_extractor(ppt_stem, aggregate_type, feat_name)
        
        # Load the mask
        mask = nib.load('%s/%s_MNI_1mm.nii.gz' % (mask_path, ROI)).get_data()            
        
        plt.figure()
        bins_all = np.arange(-10, 10.5, 0.5)
        hist_vals_all = np.zeros((len(files), len(bins_all) - 1))
        prop_sig = []
        mean_val = []
        ppts = []
        for file_counter, file in enumerate(files):

            # Load the data
            nii = nib.load(file)
            vol = nii.get_data()
            
            if aggregate_type.find('tstat') == -1:
                
                # Get the ppt name
                ppt = file[file.rfind('/s') + 1:file.find('.nii.gz')]
                if ppt.find('functional') > -1:
                    ppt = ppt[:ppt.rfind('_')] # Take all characters before the last underscore

                run_name = file[file.find('functional') + 10:file.find(feat_name) - 1]
                
            else:
                ppt = 'group'
                
            # Pull out the ppt
            ppts += [ppt]

            # Apply the mask
            vol *= mask 

            # Get just the voxels in the mask
            values = vol[vol != 0] 
            if  DV.find('proportion_sig_voxels_') >= 0:

                thresh_val = float(DV[22:])
                # Specify the threshold
                prop_sig += [np.mean(values > thresh_val)]

                #print('%0.3f' % (prop_sig[-1]))

            elif  DV == 'mean':

                # Specify the threshold
                mean_val += [np.mean(values)]

        
        # Store all of the participants names
        if ROI_counter == 0 and aggregate_type.find('tstat') == -1:
            
            # Get the participant age
            ppt_data = pd.read_csv(preloaded_path + 'participant_age.txt', header=None, sep='\t', names=['neuropipe_name', 'age', 'cohort'])

            included_ages = []
            for ppt in ppts:
                included_ages += [ppt_data[ppt_data['neuropipe_name'] == ppt]['age'].values[0]]
            
            df['Ppt_age'] = included_ages

        # Wrap up results
        if DV.find('proportion_sig_voxels_') >= 0:
            df[ROI] = prop_sig
            
        elif DV == 'mean':
            df[ROI] = mean_val            
    
    #print('Using %d/%d possible files' % (len(ppts), len(files)))
    
    # If you are returning a dictrionary then do so here
    if  DV.find('proportion_sig_voxels_') >= 0:
        
        # Return the proportion sig values
        return df
    elif  DV == 'mean':

        # Return the proportion sig values
        return df
    
    elif  aggregate_type.find('tstat') >= 0:

        # Return the proportion sig values
        return prop_sig
    
def load_each_subject_comparisons(comparisons, aggregate_type, ppt_stem, ROI, DV):
    
    # Cycle through the ROIs
    df = pd.DataFrame()
    data = {}
    ppts_all = []
    for feat_counter, feat_name in enumerate(comparisons):
    
        # Get the file names
        files = file_name_extractor(ppt_stem, aggregate_type, feat_name)
        
        # Load the mask
        mask = nib.load('%s/%s_MNI_1mm.nii.gz' % (mask_path, ROI)).get_data()            
        
        data[feat_name] = {}
        
        for file_counter, file in enumerate(files):

            # Load the data
            nii = nib.load(file)
            vol = nii.get_data()
            
            if aggregate_type.find('tstat') == -1:
                
                # Check that there are the minimum number of blocks 
                ppt_run = file[file.rfind('/s') + 1:file.find('.nii.gz')]
                
                
            else:
                ppt_run = 'group'

            # Add this participant and run name
            ppts_all += [ppt_run]

            # If this is in standard space then apply the mask
            vol *= mask 

            # Get just the voxels in the mask
            values = vol[vol != 0] 

            if  DV.find('proportion_sig_voxels_') >= 0:

                thresh_val = float(DV[22:])
                # Specify the threshold
                data[feat_name][ppt_run] = np.mean(values > thresh_val)

                #print('%0.3f' % (prop_sig[-1]))

            elif  DV == 'mean':

                # Specify the threshold
                data[feat_name][ppt_run] = np.mean(values)
    
    # Remove redundant calls
    ppts_all = np.unique(ppts_all)
    
    # Get the participant age
    ppt_data = pd.read_csv(preloaded_path + 'participant_age.txt', header=None, sep='\t', names=['neuropipe_name', 'age', 'cohort'])

    included_ages = []
    for ppt_run in ppts_all:
        
        if ppt_run.find('functional') > -1:
            ppt_run = ppt_run[:ppt_run.rfind('_')] # Take all characters before the last underscore

        included_ages += [ppt_data[ppt_data['neuropipe_name'] == ppt_run]['age'].values[0]]

    df['Ppt_age'] = included_ages
    
    # Cycle through the conditions and add the participant
    
    for condition in data.keys():
        
        feat_data = data[condition]
        
        # Cycle through the participants. If there is a match then add it, otherwise put a NAN
        feat_data_list = []
        for ppt in ppts_all:
            
            if ppt in feat_data.keys():
                feat_data_list += [feat_data[ppt]]
            else:
                feat_data_list += [np.nan]
        
        # Append the data to the list
        df[condition] = feat_data_list
                
    # Return the proportion sig values
    return df

def randomise_diff(diff_data, resample_num=10000, return_percentile=False):        
    
    # Resample the participants
    resample_diff = []
    for i in range(resample_num):
        
        # Determine what participants to use in the sample
        sub_idx = np.random.randint(0, len(diff_data), (1, len(diff_data)))

        resample_diff += [np.mean(diff_data[sub_idx])]
    
    # If specified, return the percentile
    if return_percentile is True:
        lower_bound = np.percentile(resample_diff, 2.5)
        upper_bound = np.percentile(resample_diff, 97.5)
        return lower_bound, upper_bound
        
    # What direction was the effect
    sign_count = np.sum((diff_data) > 0)
    
    # Calculate the 2 way p value
    p_val = (1 - (np.sum(np.asarray(resample_diff) > 0) / (resample_num + 1))) * 2
    
    # return the difference in ROI and 
    return p_val, sign_count

def run_randomise_stats(df):
    
    print('V1>A1')
    resampled_p_val, sign_count = randomise_diff(df['V1'].values - df['A1'].values)
    print('%d/%d show effect, p = %0.3f' % (sign_count, len(df['V1'].values), resampled_p_val))
    print('LOC>A1')
    resampled_p_val, sign_count = randomise_diff(df['LOC'].values - df['A1'].values)
    print('%d/%d show effect, p = %0.3f' % (sign_count, len(df['V1'].values), resampled_p_val))
    print('LOC>V1')
    resampled_p_val, sign_count = randomise_diff(df['LOC'].values - df['V1'].values)
    print('%d/%d show effect, p = %0.3f' % (sign_count, len(df['V1'].values), resampled_p_val))

    print('V1>0.05')
    resampled_p_val, sign_count = randomise_diff(df['V1'].values - 0.05)
    print('M = %0.2f [SD = %0.2f], %d/%d above 0.05, p = %0.3f' % (df['V1'].values.mean(), df['V1'].values.std(), sign_count, len(df['V1'].values), resampled_p_val))
    print('LOC>0.05')
    resampled_p_val, sign_count = randomise_diff(df['LOC'].values - 0.05)
    print('M = %0.2f [SD = %0.2f], %d/%d above 0.05, p = %0.3f' % (df['LOC'].values.mean(), df['LOC'].values.std(), sign_count, len(df['LOC'].values), resampled_p_val))
    print('A1>0.05')
    resampled_p_val, sign_count = randomise_diff(df['A1'].values - 0.05)
    print('M = %0.2f [SD = %0.2f], %d/%d above 0.05, p = %0.3f' % (df['A1'].values.mean(), df['A1'].values.std(), sign_count, len(df['A1'].values), resampled_p_val))

## Show wholebrain task based activation for the two cohorts<a id='wholebrain'></a>
Load in the whole brain tstat results and plot them according to the threshold. Uses the ROIs specified to trace the ROI contours and overlay them. Does this first for the runwise analyses and then the sessionwise, for both cohorts. The N for each analysis is specified in order to set the p values.

In [ ]:
threshold=0.995
whole_brain_tstat_dir = preloaded_path + '/wholebrain_tstat/'
ROIs = ['V1', 'LOC', 'A1']


In [ ]:
## Cohort 1 runwise
aggregate_type = 'runwise' 
cohort = 1
filename='%s/cohort_%d_%s.nii.gz' % (whole_brain_tstat_dir, cohort, aggregate_type)

# Get the path for this data
N = 32 # How many runs are you considering for the t test if there is one

# What is the output name for the data
output_name = '%s/cohort_%d_%s_%0.3f.svg' % (output_mask_path, cohort, aggregate_type, threshold)

plot_sig_map(filename, output_name, threshold, N, ROIs)

In [ ]:
## Cohort 2 runwise
aggregate_type = 'runwise' 
cohort = 2
filename='%s/cohort_%d_%s.nii.gz' % (whole_brain_tstat_dir, cohort, aggregate_type)

# Get the path for this data
N = 26 # How many runs are you considering for the t test if there is one

# What is the output name for the data
output_name = '%s/cohort_%d_%s_%0.3f.svg' % (output_mask_path, cohort, aggregate_type, threshold)

plot_sig_map(filename, output_name, threshold, N, ROIs)

In [ ]:
## Cohort 1 sessionwise
aggregate_type = 'sessionwise' 
cohort = 1
filename='%s/cohort_%d_%s.nii.gz' % (whole_brain_tstat_dir, cohort, aggregate_type)

# Get the path for this data
N = 14 # How many runs are you considering for the t test if there is one

# What is the output name for the data
output_name = '%s/cohort_%d_%s_%0.3f.svg' % (output_mask_path, cohort, aggregate_type, threshold)

plot_sig_map(filename, output_name, threshold, N, ROIs)

In [ ]:
## Cohort 2 sessionwise
aggregate_type = 'sessionwise' 
cohort = 2
filename='%s/cohort_%d_%s.nii.gz' % (whole_brain_tstat_dir, cohort, aggregate_type)

# Get the path for this data
N = 14 # How many runs are you considering for the t test if there is one

# What is the output name for the data
output_name = '%s/cohort_%d_%s_%0.3f.svg' % (output_mask_path, cohort, aggregate_type, threshold)

plot_sig_map(filename, output_name, threshold, N, ROIs)

## Plot bar plots of ROIs significant voxels<a id='rois'></a>
For each ROI in each participant report the number of voxels that exceed a z value corresponding to p = 0.05 (one-tailed). Plot these for each participant, coded by age.

In [ ]:
feat_name = 'default' 
p_val = 0.95 # What is the cut off you want to use
z_val = stats.norm.ppf(p_val)
DV = 'proportion_sig_voxels_%0.2f' % z_val
column_plot = 0 # Do you want to use a column plot or a bee swarm
y_range = [0.0, 0.3]
y_ticks = [0, 0.1, 0.2, 0.3] # What are the y ticks

In [ ]:
cohort_number = 1 # Cohort 1 or 2
aggregate_type = 'all' # Use sessionwise ('concat') or runwise ('all')

# Get the ROI values
df = load_each_subject(feat_name, aggregate_type, cohort_number, ['V1', 'LOC', 'A1'], DV)

plot_bar(df, [V1_color, LOC_color, A1_color])
plt.savefig(output_mask_path + 'cohort_1_%s_ROI_comparison.svg' % aggregate_type)

# Plot the line graph by age
plot_column(df)
plt.savefig(output_mask_path + 'cohort_1_%s_ROI_line.svg' % aggregate_type)

# Run all the relevant stats
run_randomise_stats(df)


In [ ]:
cohort_number = 2
aggregate_type = 'all' # Use sessionwise ('concat') or runwise ('all')

# Get the ROI values
df = load_each_subject(feat_name, aggregate_type, cohort_number, ['V1', 'LOC', 'A1'], DV)

plot_bar(df, [V1_color, LOC_color, A1_color])
plt.savefig(output_mask_path + 'cohort_2_%s_ROI_comparison.svg' % aggregate_type)

# Plot the line graph by age
plot_column(df)
plt.savefig(output_mask_path + 'cohort_2_%s_ROI_line.svg' % aggregate_type)

# Run all the relevant stats
run_randomise_stats(df) 

Show the concatenated analyses

In [ ]:
cohort_number = 1
aggregate_type = 'concat' # Use sessionwise ('concat') or runwise ('all')

# Get the ROI values
df = load_each_subject(feat_name, aggregate_type, cohort_number, ['V1', 'LOC', 'A1'], DV)

plot_bar(df, [V1_color, LOC_color, A1_color])
plt.savefig(output_mask_path + 'cohort_1_%s_ROI_comparison.svg' % aggregate_type)

# Plot the line graph by age
plot_column(df)
plt.savefig(output_mask_path + 'cohort_1_%s_ROI_line.svg' % aggregate_type)

# Run all the relevant stats
run_randomise_stats(df)
  

In [ ]:
cohort_number = 2
aggregate_type = 'concat' # Use sessionwise ('concat') or runwise ('all')

# Get the ROI values
df = load_each_subject(feat_name, aggregate_type, cohort_number, ['V1', 'LOC', 'A1'], DV)

plot_bar(df, [V1_color, LOC_color, A1_color])
plt.savefig(output_mask_path + 'cohort_2_%s_ROI_comparison.svg' % aggregate_type)

# Plot the line graph by age
plot_column(df)
plt.savefig(output_mask_path + 'cohort_2_%s_ROI_line.svg' % aggregate_type)

# Run all the relevant stats
run_randomise_stats(df)


## Explore preprocessing parameters<a id='explore'></a>

Run the preprocessing comparison analyses. For each type of feat run, this sets up a comparison (listed below) and then reports bar plots for each feat (ignoring NaNs) and plot lines connecting a run/session across conditions. 

This python code cannot not compute the statistics because special packages are needed. Instead, use the text file outputs of this (currently commented out) and refer to these files in the `stats_preprocessing.R` script

In [ ]:
comparisons = [['MotionConfounds_fslmotion_thr0.5', 'MotionConfounds_fslmotion_thr1', 'MotionConfounds_fslmotion_thr3', 'MotionConfounds_fslmotion_thr6', 'MotionConfounds_fslmotion_thr12'],
             ['default', 'Motion_recovery_1', 'Motion_recovery_2'],
             ['smoothing_0', 'smoothing_3', 'smoothing_5', 'smoothing_8'],
             ['MELODIC_thresh_0.25_fslmotion_thr3', 'MELODIC_thresh_0.5_fslmotion_thr3', 'MELODIC_thresh_1.00_fslmotion_thr3'],
             ['Temporal_derivative', 'default'],
             ['Despiking_None', 'default'],
            ]

In [ ]:
cohort_number = 1
aggregate_type = 'all' # Use sessionwise ('aggregate') or runwise ('all')

y_range = [-0.05, 0.8]
y_ticks = [0, 0.25, 0.5, 0.75] # What are the y ticks

colors = [V1_color, LOC_color, A1_color]
for ROI_counter, ROI in enumerate(ROIs):

    # Cycle through the conditions
    
    for comparison in comparisons:

        df = load_each_subject_comparisons(comparison, aggregate_type, cohort_number, ROI, DV)
        
        # Make a string of the comparisons
        comparison_str = ''
        for condition in comparison:
            comparison_str += condition + '_' 
        
        print('%s%s' % (comparison_str, ROI))
        plot_df_line(df, colors[ROI_counter])
        
        # Set the ylim depending on the type
        if 'proportion_sig_voxels' in DV:
            plt.ylim(y_range)
            plt.yticks(y_ticks)
            plt.ylabel('Prop. sig task vs rest')
            
        plt.savefig(output_mask_path + 'cohort_1_%s%s_comparison.svg' % (comparison_str, ROI))
        
        # Save data for R stats
        df.to_csv(output_mask_path + 'cohort_1_%s%s_comparison.txt' % (comparison_str, ROI), index=False, na_rep='NaN')
        

Run this on the concat data

In [ ]:
cohort_number = 1
aggregate_type = 'concat' # Use sessionwise ('concat') or runwise ('all')

y_range = [-0.05, 1]
y_ticks = [0, 0.25, 0.5, 0.75, 1.0] # What are the y ticks

colors = [V1_color, LOC_color, A1_color]
for ROI_counter, ROI in enumerate(ROIs):

    # Cycle through the conditions
    
    for comparison in comparisons:

        df = load_each_subject_comparisons(comparison, aggregate_type, cohort_number, ROI, DV)
        
        # Make a string of the comparisons
        comparison_str = ''
        for condition in comparison:
            comparison_str += condition + '_' 
        
        print('%s%s' % (comparison_str, ROI))
        plot_df_line(df, colors[ROI_counter])
        
        # Set the ylim depending on the type
        if 'proportion_sig_voxels' in DV:
            plt.ylim(y_range)
            plt.yticks(y_ticks)
            plt.ylabel('Prop. sig task vs rest')
    
        plt.savefig(output_mask_path + 'cohort_1_concat_%s%s_comparison.svg' % (comparison_str, ROI))
        
        # Save data for R stats
        df.to_csv(output_mask_path + 'cohort_1_concat_%s%s_comparison.txt' % (comparison_str, ROI), index=False, na_rep='NaN')
        